In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import math 
import time
import graph
import numpy as np
from collections import Counter
from tqdm import tqdm
import array as arr
import copy

#filepathTXT = "../edgelists/BlogCatalog-edgelist.txt"
filepathCSV = "input/BlogCatalog-edgelist.csv"
#embeddingsrecursive = "../embeddings/BlogCatalog-edgelist.txt.embeddings-recursive"
#embeddingsiterative = "../embeddings/BlogCatalog-edgelist.txt.embeddings-iterative"

In [2]:
def parseEdgeList2(graph_file, direction = "undirected"):
    # Create Graph
    G = nx.Graph()
    # Create head
    colNames=["Start", "End"]
    edgeData = pd.read_csv(filepathCSV, names=colNames)

    #Add nodes
    nodes = []
    #loop throug data records
    for i in range (0, edgeData.shape[0]):
        #append every node
        nodes.append(edgeData.iloc[i,0])
        nodes.append(edgeData.iloc[i,1])
    #creating a set of nodes    
    nodes = set(nodes)
    #sorting the nodes in increasing order
    uniqueNodes = (list(nodes))
    uniqueNodes.sort()
    #adding the nodes to the graph
    G.add_nodes_from(uniqueNodes)

    # Add edges
    #loop from 0 to amount of records
    edgeCount = 0
    for i in range (0, edgeData.shape[0]):
        edgeCount += 1
        #add the edge to the graph
        G.add_edge(edgeData.iloc[i,0], edgeData.iloc[i,1])
    print("Nodes: ", G.number_of_nodes()," Edges: ", G.number_of_edges(), " loaded from ", graph_file)

    if(direction == "undirected"):
        return G.to_undirected()
    else:
        return G

In [3]:
def getAdjNPList(graph):
    adjdict = {}
    for vertex in graph:
        adjdict[vertex] = np.array([n for n in G.neighbors(vertex)]) 
        np.random.shuffle(adjdict[vertex])
    return adjdict

def getNodeContextSets(graph):
    all_sets = {}
    for node in graph:
        all_sets[node] = []
    return all_sets

In [4]:
def CountContextPairs(contextPairs):
    countSum = 0
    for key, value in contextPairs.items():
        countSum += value
    print("Total value sums up to: ", countSum)
    

In [ ]:
#femb_recursive = open(embeddingsrecursive, 'w')
#femb_iterative = open(embeddingsiterative, 'w')

## Toy Graph

In [ ]:
def getToyGraph():
    G = nx.Graph()
    G.add_nodes_from(["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34"])
    inputfile = "input/karate.adjlist"
    lines = open(inputfile, "r")
    for line in lines:
        line = line.split()
        for node in line[1:]:
            G.add_edge(line[0], node);
    print("Nodes: ", G.number_of_nodes()," Edges: ", G.number_of_edges())
    #print("G.nodes ->", G.nodes)
    # Draw graph
    #nx.draw(G, with_labels = True)
    #plt.show()
    return G


### Parametrs

In [5]:
# Set the actual parameters and graph
WALK_LENGHT = 40
BUDGET = 80
ORIGINAL_WINDOW_SIZE = 10

EPSILON = 0.5
WINDOW_SIZE = ORIGINAL_WINDOW_SIZE*2+1
QUEUE_BUFFER_SIZE = (BUDGET*WALK_LENGHT)-(BUDGET*2) + 1
#G = parseEdgeList1(filepathTXT) #String labels 

G = parseEdgeList2(filepathCSV) # Integer labels
adjdict = getAdjNPList(G)
all_sets = getNodeContextSets(G)

Nodes:  10312  Edges:  333983  loaded from  input/BlogCatalog-edgelist.csv


In [ ]:
# Set toy parameters and graph
WALK_LENGHT = 5
BUDGET = 2
ORIGINAL_WINDOW_SIZE = 1

EPSILON = 0.5
WINDOW_SIZE = ORIGINAL_WINDOW_SIZE*2+1
QUEUE_BUFFER_SIZE = (BUDGET*WALK_LENGHT)-(BUDGET*2) + 1

G = getToyGraph()
adjdict = getAdjNPList(G)
all_sets = getNodeContextSets(G)

### NP implementation

In [6]:
def updateSetCount(startvertex, context_pair, context_budget):
    for i in range(0, context_budget):
        all_sets[startvertex].append(context_pair)

def chooseNodes(list_nodes, sample_size):
    random.seed(0)
    return random.sample(population=list_nodes, k=sample_size) # 2.22 s ± 179 ms per loop  (labesl str)

def getPerNodeBudget(numNodes, budget):
    return math.floor(budget/numNodes)

def storeContextPairs(context_pair, budget, context_pairs):
    if context_pair not in context_pairs:
        context_pairs[context_pair] = budget
    else:
        context_pairs[context_pair] = context_pairs[context_pair] + budget
        
def updateContextPairs(startvertex, window, window_count, context_pairs):
    lastNode = window[window_count]
    labelOfLastNode, budgetOfLastNode = lastNode
    index_count = 0
    window_except_last = window[:window_count]
    window_reversed = window_except_last[::-1]
    for node in window_reversed:
        node_label = node[0]
        if index_count == ORIGINAL_WINDOW_SIZE:
            break
        context_pair1 = (str(labelOfLastNode) +","+ str(node_label))
        context_pair2 = (str(node_label) + "," + str(labelOfLastNode))
        storeContextPairs(context_pair1, budgetOfLastNode, context_pairs)
        storeContextPairs(context_pair2, budgetOfLastNode, context_pairs)
        index_count = index_count + 1
        
        updateSetCount(startvertex, context_pair1, budgetOfLastNode)
        updateSetCount(startvertex, context_pair2, budgetOfLastNode)


def addNewNodeToWindow(tempwindow, temp_window_count, window_size, vertex, budget):
    newWindowElement = np.array([[vertex,budget]])
    if temp_window_count+1 == window_size:
        tempwindow = tempwindow[1:]
        tempwindow[temp_window_count] = newWindowElement
        return tempwindow, temp_window_count

    tempwindow[temp_window_count+1] = newWindowElement
    temp_window_count+=1
    return tempwindow, temp_window_count

def BFSRandomWalkWindow(startvertex, queue, queue_len, queue_pop_idx, queue_add_idx, context_pairs, window_size, walk_lenght):
    queue_buffer_size = (queue.size/5)-1
    while queue_len > 0:
        vertex, budget, current_walk_lenght, window_count, window = queue[queue_pop_idx] 

        if all_sets[vertex] and vertex != startvertex:
            current_amount_of_cp = len(all_sets[startvertex])
            needed_amount_cp = len(all_sets[1])
            cp_to_sample = needed_amount_cp - current_amount_of_cp
            sampled_cp = random.sample(all_sets[vertex], cp_to_sample)
            for cp in sampled_cp:
                storeContextPairs(cp, 1, context_pairs) 
            all_sets[startvertex].extend(sampled_cp)
            break
        
        queue_len -=1
        if queue_len > 0:
            if queue_pop_idx == queue_buffer_size:
                queue_pop_idx = 0
            else:
                queue_pop_idx += 1
        else:
            queue_add_idx = 0  
    
        vertex_neighbors = adjdict[vertex]
        num_neighbors = vertex_neighbors.size
        per_node_budget = getPerNodeBudget(num_neighbors, budget)
        remainder = budget - (per_node_budget * num_neighbors)
        
        bonus = 0
        if remainder > 0:
            np.random.shuffle(vertex_neighbors) 
            bonus = remainder 
            
        current_walk_lenght += 1
        for neighbor in vertex_neighbors:
    
            budget_for_this_node = per_node_budget 
            temp_window = np.copy(window)
            temp_window_count = window_count 
            
            if bonus == 0 and budget_for_this_node == 0:
                break
             
            if bonus > 0:    
                bonus -= 1
                budget_for_this_node += 1          
                     
            temp_window, temp_window_count = addNewNodeToWindow(temp_window, temp_window_count, window_size, neighbor, budget_for_this_node)
            updateContextPairs(startvertex, temp_window, temp_window_count, context_pairs) 
            if current_walk_lenght < walk_lenght:
                newQueueElement = np.array([[neighbor, budget_for_this_node, current_walk_lenght, temp_window_count, temp_window]])
                queue[queue_add_idx] = newQueueElement
                queue_len+=1
                if queue_add_idx == queue_buffer_size:
                    queue_add_idx = 0
                    continue
                queue_add_idx += 1  

## NP Runner

In [7]:
def Runner():
    start = time.time()
    context_pairs = {}    
    print("Running BFSRandomWalkWindow...")

    for startvertex in adjdict.keys():
        
        #print("")
        #print("Running from -> ", startvertex, "with budget ->", BUDGET)
        
        window_count = 0
        WINDOW = np.zeros(shape=(WINDOW_SIZE+20,2),dtype=int)
        firstWindowElement = np.array([[startvertex, BUDGET]])
        WINDOW[window_count] = firstWindowElement

        queue_len = 0
        queue_pop_idx = 0
        queue_add_idx = 0
        queue = np.zeros(shape=(BUDGET,5),dtype=object)
        firstQueueElement = np.array([[startvertex, BUDGET, 1, window_count, WINDOW]])
        queue[queue_len] = firstQueueElement
        queue_len+=1
        queue_add_idx+=1

        BFSRandomWalkWindow(startvertex, queue, queue_len, queue_pop_idx, queue_add_idx, context_pairs, WINDOW_SIZE, WALK_LENGHT)
    
    end = time.time()
    result = end - start
    print("Run in :",result)
    return context_pairs

## Run

In [8]:
contextPairs = Runner()

Running BFSRandomWalkWindow...
Run in : 1607.4035108089447


In [9]:
# Count the total count sum
CountContextPairs(contextPairs)

Total value sums up to:  569222400


### Writing to file

In [ ]:
print("Writing context pairs to file...")    
#Writing to file    
for (key, value) in contextPairs.items():
    femb_iterative.write(str(key) + " " + str(value) + "\n" )
femb_iterative.close()

In [21]:
def storeContextPairs(context_pair, budget, context_pairs):
    if context_pair not in context_pairs:
        context_pairs[context_pair] = budget
    else:
        context_pairs[context_pair] = context_pairs[context_pair] + budget

In [22]:
lis = [{"a":4, "b":5},{"a":1, "b":3, "f":7},{"a":4, "b":5, "c":4}]

In [23]:
def CountDictListCp(lis):
    contextPairs = {}
    for dicset in lis:
        for CP,budget in dicset.items():
            storeContextPairs(CP,budget,contextPairs)
    return contextPairs
        
cp = CountDictListCp(lis)
cp
    

{'a': 9, 'b': 13, 'f': 7, 'c': 4}

In [ ]:
from multiprocessing import shared_memory
shm_a = shared_memory.SharedMemory(create=True, size=10)
type(shm_a.buf)
<class 'memoryview'>
buffer = shm_a.buf
len(buffer)
10
buffer[:4] = bytearray([22, 33, 44, 55])  # Modify multiple at once
buffer[4] = 100                           # Modify single byte at a time
# Attach to an existing shared memory block
shm_b = shared_memory.SharedMemory(shm_a.name)
import array
array.array('b', shm_b.buf[:5])  # Copy the data into a new array.array
array('b', [22, 33, 44, 55, 100])
shm_b.buf[:5] = b'howdy'  # Modify via shm_b using bytes
bytes(shm_a.buf[:5])      # Access via shm_a
b'howdy'
shm_b.close()   # Close each SharedMemory instance
shm_a.close()
shm_a.unlink()  # Call unlink only once to release the shared memory

In [24]:
from multiprocessing import shared_memory
shm_a = shared_memory.SharedMemory(create=True, size=10)
type(shm_a.buf)

ImportError: cannot import name 'shared_memory' from 'multiprocessing' (C:\Users\Max\Anaconda3\lib\multiprocessing\__init__.py)

In [4]:
random.sample([1,2,3,3,3,3], 3)

[3, 1, 3]